
<img src="../images/starwarsstartrek.png" style="float: center; margin: 0px; height: 150px; width: 500px"> 

## SUBREDDIT CLASSIFIER

###### _Author: Afolabi Cardoso_

---

### DATA COLLECTION

---
#### Contents:
[Overview](#Overview) | [Imports](#Imports) | [Data Cleaning Training Set](#Data-Cleaning-Training-Set) | [Data Cleaning Test Set](#Data-Cleaning-Test-Set) | [Exports](#Exports)


---
#### Overview



#### Imports

In [8]:
import pandas as pd
import numpy as np
import requests
import time

#### Data Collecting`

I am going to create a **get_post** function. The function accepts two parameters. The first parameter is a list containing the names of the subreddits and the second parameter is the number of post I want to collect.

The functions returns a list of dataframes containing all the requested posts.

In [9]:
#function definition get_post

def get_post(lst, no_post):
    url = 'https://api.pushshift.io/reddit/search/submission' #pushshift reddit API link   
    posts = []
    
    for sub in lst:
        post_count = no_post   #assign the number of post passed to post_count
        before  = int(time.time())  #get the current time and assigns it to before
        
        while post_count > 0:
                
            if post_count < 100:   
                size = post_count
            else:
                size = 100
            
            params = {                      #parameters for the get method
                'subreddit': sub,
                 'size' : size,
                 'before': before
             }

            res = requests.get(url, params)
            json_data = res.json()
            df_data = pd.DataFrame(json_data['data'])
            
            posts.append(df_data)
            
            post_count -= 100 
            before = df_data['created_utc'].iloc[-1]
            
        time.sleep(3)  #create a 3 seconds wait between pulls
    return posts

I am going to call the **get_post** and pass in the subreddits and number of posts. I will collect 1010 post, the extra 10 post is for if I need to replace posts with errors.

In [10]:
subreddits = ['StarWars', 'startrek']
df_list = get_post(subreddits, 1002)

Concat the list of dataframes

In [11]:
df = pd.concat(df_list, ignore_index=True)

In [23]:
df.shape

(2002, 84)

Export the raw dataframe as raw.csv

In [21]:
df.to_csv('../data/raw.csv', index=False)